# Working with BBM database

This notebook illustrates the capability of AEON.py to fetch and prepare BN models directly from the Biodivine Boolean Models database.

*Work in progress...*

In [1]:
from biodivine_aeon import *

You can fetch all model IDs from the BBM API or query for a BN with a given ID. At the moment, the model ID is a string ID used by the API, which is not the same as the numerical id clasically used in the BBM database. This is a temporary solution until the BBM endpoint is updated to provide the numerical id as well.

Let's first request the list of possible IDs. The fetch operations may take a few seconds, as we are accessing the external database endpoint.

In [2]:
ids = BiodivineBooleanModels.fetch_ids()
ids[:5]  # Show the first 5 ids

['202fec2d-a27d-45cd-a1d2-6d619412b1d5',
 '15ce1709-b98e-4fcf-9183-9adda842a437',
 'dad447c8-c14d-40d2-8496-d8953690a20a',
 '6454b139-57fa-4a7f-8f52-5623db697dc5',
 '6bfb7209-57ea-40e1-9d78-cbde5f9d106f']

Let's now fetch a particular Boolean network querrying its ID.

In [ ]:
# Fetch a small network (with 18 variables, 8 of which are inputs) by its ID
bn = BiodivineBooleanModels.fetch_network("b427a7d3-5158-48ab-991b-1db895dd2b4d")
print(bn)

BooleanNetwork(variables=18, regulations=18, explicit_parameters=0, implicit_parameters=8)


In [4]:
print(bn.to_aeon())

v_Pvr -> v_Drk
v_Raf -> v_Dsor1
v_CNK -> v_Raf
v_Ksr -> v_Raf
v_Src42 -> v_Raf
v_Ras -> v_Raf
v_Rl -> v_Pnt
v_Drk -> v_Sos
v_Pvf2 -> v_Pvr
v_Pvf3 -> v_Pvr
v_Pvf1 -> v_Pvr
v_Dsor1 -> v_Rl
v_msk -> v_Rl
v_Sty -| v_Ras
v_Sos -> v_Ras
v_Aop -| v_Targets
v_Pnt -> v_Targets
v_Rl -| v_Aop
$v_Aop: !v_Rl
$v_Drk: v_Pvr
$v_Dsor1: v_Raf
$v_Pnt: v_Rl
$v_Pvr: v_Pvf2 | v_Pvf1 | v_Pvf3
$v_Raf: v_Ras & v_CNK & v_Ksr & v_Src42
$v_Ras: v_Sos & !v_Sty
$v_Rl: v_Dsor1 & v_msk
$v_Sos: v_Drk
$v_Targets: v_Pnt & !v_Aop

